1. PCA

In [ ]:
# train
from sklearn.decomposition import PCA
pca = PCA(n_components = components)
pca.fit(X)

# test_detection
data_test_pca = pca.transform(Y)
data_pred = pca.inverse_transform(data_test_pca)

2. UAE

In [ ]:
class UAE(nn.Module):
    def __init__(self, image_size = 36 * n):
        super(UAE, self).__init__()
        self.inflate1 = nn.Sequential(
            nn.Linear(image_size, image_size * 2), 
            nn.Tanh(), )
        self.encode = nn.Sequential(
            nn.Linear(image_size * 2, 18), )
        self.inflate2 = nn.Sequential(
            nn.Linear(18, image_size * 2), 
            nn.Tanh(), )
        self.decode = nn.Sequential(
            nn.Linear(image_size * 2, image_size), )
    
    def forward(self, x):
        x = self.inflate1(x)
        x = self.encode(x)
        x = self.inflate2(x)
        x = self.decode(x)
        return x

3. 1D-CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self, classes = num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(50, 64, kernel_size = 5, stride = 1),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace = True), 
            nn.MaxPool1d(kernel_size = 2, stride = 2), )
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size = 3, stride = 1),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace = True), 
            nn.MaxPool1d(kernel_size = 2, stride = 2), )
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size = 2, stride = 1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace = True), 
            nn.MaxPool1d(kernel_size = 2, stride = 2), )
        self.conv4 = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size = 2, stride = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace = True), 
            nn.MaxPool1d(kernel_size = 2, stride = 2), )
        self.conv5 = nn.Sequential(
            nn.Conv1d(512, 1024, kernel_size = 1, stride = 1),
            nn.BatchNorm1d(1024),
            nn.MaxPool1d(kernel_size = 1, stride = 1), )

        self.classifier = nn.Sequential(
            nn.Linear(1024 * 1 * 1, 512),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.4),
            nn.Linear(512, 256),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.4),
            nn.Linear(256, classes), )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.view(x.size(0), 1024 * 1 * 1)
        x = self.classifier(x)
        return x

4. AADS

In [ ]:
class AADS(nn.Module):
    def __init__(self,m=51,w=60):
        super(AADS, self).__init__()
        self.deep = nn.Sequential(
            nn.Linear(w, 180),
            nn.Conv1d(51, 64, kernel_size=2, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(64, 128, kernel_size=2, stride=1),
            nn.MaxPool1d(kernel_size=2),
            nn.ReLU(inplace=True)
        )
        self.dl3 = nn.Linear(128,4)
        self.wide = nn.Linear(w,4)
        self.dl4 = nn.Linear(95,80)
        self.outputlayer = nn.Sequential(
            nn.Linear(80,65),
            nn.ReLU(inplace=True),
            nn.Linear(65, 50),
            nn.ReLU(inplace=True),
            nn.Linear(50, 25),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        deep_out = self.deep(x)
        deep_out = deep_out.permute(0,2,1)
        deep_out = self.dl3(deep_out)
        wide_out = self.wide(x)
        out = torch.concat([wide_out,deep_out],dim=1)
        out = out.permute(0,2,1)
        out = self.dl4(out)
        out = self.outputlayer(out)
        return out

5. DAICS

In [ ]:
# The detection model is similar to AADS, and the threshold model is as follows.
class TTNN(nn.Module):
    def __init__(self,m=51,w=60):
        super(TTNN, self).__init__()
        self.CL1 = nn.Conv1d(1,2,kernel_size=2)
        self.MP1 = nn.MaxPool1d(2)
        self.CL2 = nn.Conv1d(2,4,kernel_size=2)
        self.MP2 = nn.MaxPool1d(2)
        self.CL = nn.Linear(14,1)
    def forward(self, x):
        out =self.CL1(x)
        out = self.MP1(out)
        out = self.CL2(out)
        out = self.MP2(out)
        out = self.CL(out)
        return out